In [3]:
from transformers import pipeline
import torch 

device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float32 if torch.cuda.is_available() else torch.float16
gemma = "google/gemma-2-2b-it"
nlp_model = pipeline("text-generation", model=gemma, model_kwargs={"torch_dtype": torch_dtype}, device=device)


Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.59s/it]
Device set to use cpu


NATURAL LANGUAGE QUERYING

In [3]:
query = "display contact with foreign prefix numbers"
messages = [
    {"role": "user", "content": 
        f"""
        Context: You are a SQL generator. 
        Given the following database schema:
            Table: Messages
            Columns:
            - id (integer)
            - Address (text)
            - Date Sent (date)
            - Date Received (date)
            - Type (text)
            - Body (text)
            - Seen (boolean)
            
            Table: Contacts
            Columns:
            - id (integer)
            - name (text)
            - number (number)
            - email (text)
            
            Table: Call Logs
            Columns:
            - id (integer)
            - Owner (text)
            - Date Time (number)
            - Duration (number)
            - Type (text)
            
            Table: Files
            Columns:
            - path (text)
            - name (text)
            - parent (text)
            - size (number)
            - datetime (datetime)
            - ext (text) alias type
            
        Convert the following user question into a correct, safe SQL query. 
        Return only SQL, no explanations.
        Query: create a query to {query}
        """}
]

outputs = nlp_model(messages, max_new_tokens=128)

assistant_response = outputs[0]["generated_text"][-1]["content"].strip()
print(assistant_response)


```sql
SELECT c.name, c.number
FROM Contacts c
WHERE c.number LIKE '0%';
```


TEXT CLASSIFICATION

In [ ]:
text = "Oh I’m shaking. You talk big but all I see is someone desperate to look tough. Say whatever you want. You’re about to learn that I don’t back down."
candidate_labels = [
    "Normal", "Threat", "Identity Attack", "Sexually Explicit", "Toxicity", "Extremism", "Scam"
]

messages = [
    {
        "role": "user", "content": 
        f"""
        Text: {text}
        Query: classify the text into one of the following categories: {candidate_labels}. No explaination.
        """
    }
]

outputs = nlp_model(messages, max_new_tokens=128)
assistant_response = outputs[0]["generated_text"][-1]["content"].strip()
print(assistant_response)

The text most accurately falls into the category of **Toxicity**. 

Here's why:

* **Hostility and Aggression:** The speaker uses aggressive language like "shaking," "talk big but all I see is someone desperate to look tough," and "don't back down."  
* **Defensiveness and Deflection:** The speaker is not interested in a genuine conversation and instead is trying to dismiss the other person's points by focusing on their perceived lack of strength. 
* **Threatening Language:** The threat is implicit but clear – the speaker is implying that they will not back down and will likely react


SUMMARIZATION

In [ ]:
text = ""
messages = [
    {"role": "user", "content": 
        f"""
        {text}
        Query: Summarize this
        """}
]

outputs = nlp_model(messages, max_new_tokens=128)
print(outputs)
assistant_response = outputs[0]["generated_text"][-1]["content"].strip()
print(assistant_response)

CONVERT TO NLP

In [ ]:
import sqlite3

DB_LOCATION = "./data/"
id = "sms.db"

def run_query(query):
    if str(query).lower().startswith("select"):
        return
    conn = sqlite3.connect("./data/sms.db")
    cur = conn.cursor()
    output = cur.execute(
        query,
        []
    ).fetchall()
    return output

results = run_query(query)
print(results)

messages = [
    {
        "role": "user", 
        "content": 
        f"""
        Data: {results}
        Convert the following data in human readable format
        """
    }
]
try:
    outputs = nlp_model(messages, max_new_tokens=256)
    assistant_response = outputs[0]["generated_text"][-1]["content"].strip()
    print(assistant_response)
    
except Exception as e:
    print(e)

TESTING

In [2]:
query = "create a sql query to list all audio files"
messages = [
    {"role": "user", "content": 
        f"""
        Context: You are a SQL generator. 
        Given the following database schema:
            Table: Messages
            Columns:
            - id (integer)
            - Address (text)
            - Date Sent (date)
            - Date Received (date)
            - Type (text)
            - Body (text)
            - Seen (boolean)
            
            Table: Contacts
            Columns:
            - id (integer)
            - name (text)
            - number (number)
            - email (text)
            
            Table: Call Logs
            Columns:
            - id (integer)
            - Owner (text)
            - Date Time (number)
            - Duration (number)
            - Type (text)
            
            Table: Files
            Columns:
            - path (text)
            - name (text)
            - parent (text)
            - size (number)
            - datetime (datetime)
            - ext (text) alias type
            
        Convert the following user question into a correct, safe SQL query. 
        Return only SQL, no explanations.
        Query: create a query to {query}
        """}
]

try:
    outputs = nlp_model(messages, max_new_tokens=256)
    assistant_response = outputs[0]["generated_text"][-1]["content"].strip()
    print(assistant_response)
    
except Exception as e:
    print(e)

```sql
SELECT * FROM Files WHERE ext ='mp3' OR ext = 'wav';
```
